In [1]:
from flask import Response
from flask import Flask
import cv2
import threading

In [2]:
app = Flask(__name__)

In [3]:
vs = cv2.VideoCapture('http://217.152.196.254/axis-cgi/mjpg/video.cgi')

In [4]:
frameOut = None
lock = threading.Lock()

In [5]:
def generate():
    
    global frameOut, lock
    
    while True:
        with lock:
            if frameOut is None:
                continue 
        
            (flag, encodedImage) = cv2.imencode(".jpg", frameOut)
            frameOut = None

            if not flag:
                continue

        yield (b'--frame\r\n' b'Content-Type: image/jpeg\r\n\r\n' +
               bytearray(encodedImage) + b'\r\n')

In [6]:
def updateFrame():

    global frameOut, vs

    while True:
        frame = vs.read()  
        if (frame[0]):
            frame = frame[1]          
            with lock:
                frameOut = frame.copy()

@app.route("/")
def video_feed():
    return Response(generate(),
                    mimetype="multipart/x-mixed-replace; boundary=frame")

In [ ]:
t = threading.Thread(target=updateFrame)
t.daemon = True
t.start()
app.run(host="127.0.0.1", port="8080")

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8080/ (Press CTRL+C to quit)
127.0.0.1 - - [03/Jun/2020 12:55:28] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2020 12:55:45] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2020 12:58:48] "GET / HTTP/1.1" 200 -
